In [1]:
# Importing the required libraries.
from pymongo import MongoClient
import os
from IPython.display import clear_output
import pickle 
import zlib

# Creating the connection to the gem2 server.
client = MongoClient()

In [2]:
# Forming the location dictionary : 'locations'.
file = open('locations.csv')
locations = file.read().split('\n')[:-1]
file.close()
locations[0] = locations[0].split(',')
for i in range(1,len(locations)):
    temp = locations[i].split(',')
    locations[i] = [temp[1],int(temp[0])]
locations = dict(locations[1:])

In [3]:
# Collect article data.
def collect_articles_data(collection_name):
    
    source_collection = client['MTP'][collection_name]
    articles = source_collection.find({})
    data1, data2 = [], []
    
    processed = 0
    for article in articles:
        flag = False
        for entity in article['entities']:
            if entity['type']=='ProvinceOrState' or entity['type']=='City':
                if entity['name'].lower() in locations:
                    flag=True
                    data1.append([str(article['_id']), str(locations[entity['name'].lower().strip()])])
        if flag:
            date = str(article['publishedDate'])
            date = (date[:4],date[5:7],date[8:])
            data2.append([str(article['_id']), str(article['articleTitle']), str(article['text']), date])
        processed+=1
        print(collection_name.capitalize(), processed)
        clear_output(wait=True)

    path_name = './Collections/'+collection_name.capitalize()+' Data/'
    os.mkdir(path_name)
    
    file1 = open(path_name+'file1','wb')        # article_id, locations
    file2 = open(path_name+'file2','wb')        # article_id, title, text, date
    pickle.dump(zlib.compress(pickle.dumps(data1),pickle.HIGHEST_PROTOCOL),file1,pickle.HIGHEST_PROTOCOL)
    pickle.dump(zlib.compress(pickle.dumps(data2),pickle.HIGHEST_PROTOCOL),file2,pickle.HIGHEST_PROTOCOL)
    file1.close()
    file2.close()

In [4]:
os.mkdir('./Collections/')
for collection in ['agriculture','development','environment','industrialization','lifestyle']:
    collect_articles_data(collection)

Lifestyle 233816
